# Proyecto 1: Análisis de sentimiento (Reviews de rotten tomatoes)
En este proyecto utilizaremos el Rotten tomatoes dataset  el cual contiene 8530 reseñas de tipo string con hasta 267 caracteres, y 1066 reseñas en validación. Tu trabajo es predecir si la reseña se expresa positiva o negativamente para las que se encuentren en el conjunto de validación de la mejor manera posible, utilizando la experiencia pasada dada en las 8530 reseñas de entrenamiento. Además, analizarás la eficacia de tu modelo y evaluarás si el modelo ha aprendido correctamente.

A diferencia de los ejercicios anteriores donde programaste las soluciones analíticas a los métodos de ML, en este proyecto se recomienda el uso de las funciones y clases integradas de scikit-learn. Para entender el uso de estas clases y ver algunos ejemplos puedes consultar la documentación oficial [sk-learn user guide](https://scikit-learn.org/stable/supervised_learning.html)

En este proyecto tendrás que elegir que método de reducción de dimensionalidad y que método de agrupamiento deseas aplicar a tus datos. Es tu trabajo analizar la información dada para tomar estas decisiones. Lee con atención todas las instrucciones y celdas de código, y recuerda agregar tu código en todas las partes donde veas la instrucción "`TODO`"

## Descripción:
1. Dado que los datos están en forma de texto, será necesario transformarlos a un vector de números
2. Aplicar un método de reducción de dimensionalidad y visualizar los datos
3. Buscar grupos en los datos reducidos con alguna técnica de agrupamiento o clasificación.
4. Interpretar los resultados.
5. Dados los textos de validación, identificar a que grupo o clase pertenece. (Inferencia)

## Instrucciones
En este proyecto también tienes la alternativa de correr el entrenamiento completando el archivo de training.py y preprocessing.py, sin embargo se espera entreges las mismas gráficas y resultados independientemente de si usas el notebook o los archivos de python tradicionales
- Si utilizas el archivo de training.py, en preprocessing.py completa el método get_one_hot_vector
- De otra forma, si utilizas solo este archivo, completa el código para preprocesar los datos, entrenar y evaluar el modelo.

Nota como existen múltiples soluciones a este problema. La decisión de como resolverlo es tuya (: intenta hacerlo lo mejor posible!
Comenzamos por importar las librerías correspondientes.

En tu blog no olvides incluir:
- Análisis del conjunto de datos en baja dimensionalidad
- Evaluación cualitativa y cuantitativa (en validación) de al menos dos modelos/métodos de aprendizaje de máquina
    - Reportar métricas de clasificación para cada uno 
    - Dos ejemplos de falsos poitivos
    - Dos ejemplos de falsos negativos
    - Dos ejemplos de verdaderos positivos



# 1. Análisis del conjunto de datos

In [ ]:
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt

dataset = load_dataset('./rotten_tomatoes_dataset.py')
training_set = dataset['train']
validation_set = dataset['validation']


In [ ]:
def print_samples(dataset, n_samples, random=True):
    if random:
        indices = np.random.randint(0, len(dataset), n_samples)
    else:
        indices = np.arange(n_samples)

    for i in indices:
        idx = i.item()
        datapoint = dataset[idx]
        text = datapoint['text']
        label = datapoint['label']
        is_pos = "positive" if label else "negative"
        print(f"({is_pos}) - Text: {text}")

In [ ]:
# Carga de datos
print("# datos de entrenamiento", training_set.shape)
print("# datos de validación", validation_set.shape)
print("Muestras de entrenamiento")
print_samples(training_set, 5)

# Preprocesamiento de datos

In [ ]:
def get_one_hot_vector(text, vocabulary):
    ''' # TODO
        Dado un texto y un vocabulario, devuelve un vector one-hot
        donde el valor sea 1 si la palabra esta en el texto y 0 en caso contrario.
        Ejemplo:
            text = 'hola mundo'
            vocabulary = {
                'hola': 0,
                'mundo': 1,
                'UNK': 2
            }
            one_hot = [1, 1, 0]
    '''
    # Inicializamos un vector en ceros
    embedded = np.zeros(len(vocabulary))
    # TODO: Modifica los valores segun la descripción
    # Si lo deseas, puedes experimentar para que el vector
    # contenga la cantidad de veces que aparece la palabra
    # en lugar de solo indiccar si existe o no,

    return embedded

El siguiente método llama al anterior para preprocesar el conjunto de datos completo

#todo: Investiga sobre Word2Vec e incluye lo que encuentres al respecto en tu blog. ¿Sería una mejor manera de representar el texto?¿Que sería necesario para poder usarlo?

In [ ]:
from preprocessing import get_vocab
def preprocess_dataset(dataset, vocabulary):
    '''
        Datado un dataset (x, y) donde x es un texto y y es la etiqueta,
        devuelve una matriz X donde cada fila es un vector one-hot
        y un vector y con las etiquetas.
        Ejemplo:
            vocab = {"hola": 0, "mundo": 1, "cruel": 2}
            input: 
            dataset = [
                       {"text": "hola mundo cruel", "label": 0},
                       {"text": "hola mundo", "label": 1}
                       ]
            output:
            X = [[1, 1, 1],
                 [1, 1, 0]]
    '''
    X = []
    y = []
    for i in range(len(dataset)):
        sample = dataset[i]
        X.append(get_one_hot_vector(sample['text'], vocabulary))
        y.append(sample['label'])
    return np.array(X), np.array(y)

# Tanto en entrenamiento como validación, obtenemos el vocabulario del conjunto de entrenamiento
data_train, target_train = preprocess_dataset(training_set, get_vocab(training_set))
data_val, target_val = preprocess_dataset(validation_set, get_vocab(training_set))

Después de transformar el texto a modo vector usando la representación anterior, la dimensionalidad de nuestros datos habra cambiado bastante.

In [ ]:
print("Datos de entrenamiento", data_train.shape, target_train.shape)
print("Datos de validación", data_val.shape, target_val.shape)

## Visualización en baja dimensionalidad
En la siguiente celda puedes visualizar como se ven tus datos reduciendo la dimensionalidad a 2. Explora usar TSNE y PCA, y elige el que te de mejor información. En este caso debido a que la dimensionalidad es demasiado alta, puedes intentar reducir la dimensionalidad en 2 etapas. Por ejemplo, usando pca para reducir los datos inicialmente a 100-300 variables, y consecuentemente usar PCA o TSNE para reducir a 2 o 3 dimensionsiones para visualizacion

Utiliza las siguientes preguntas para analizar la imgen:
- ¿Cual método de reducción de dimensionalidad funciona mejor en este caso?
- ¿Que puedes deducir de esta imagen?
- ¿Que ocurre si intentas reducir la dimensionalidad original a 2d en una sola etapa?
- ¿Cuál método de redicción es mas eficiente?
- ¿Qué representa cada color en este caso?

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# TODO Reducimos la dimensionalidad de los datos de validacion data_val
# a 2 dimensiones usando TSNE y/o PCA
reduced_data = 

labels = np.unique(target_train)
fig, ax_pca = plt.subplots(1, 1, figsize=(4,4))
fig.suptitle("Puntos reducidos a dos dimensiones")
for c in labels:
    indices = np.where(target_train == c)
    plot_data = reduced_data[indices]
    ax_pca.scatter(plot_data[:, 0], plot_data[:, 1], label=f"Grupo {c}")
plt.show()

# 2. Tu turno! - Entrenamiento
Utiliza los datos `data_train` con las etiquetas `target_train` define para entrenar un modelo que identifique dígitos. 
Utiliza las librerías de sklearn para entrenar al menos dos modelos. En esta sección queda a tu criterio:
- Decidir si entrenarás en alta o baja dimensionalidad
- Decidir los modelos que deseas comparar
- Decidir si quieres usar un método de aprendizaje supervisado o no supervisado, o comparar ambos.
- tip: Investiga si existen métdos de aprendizaje de máquina tradicionales que se apliquen principalmente a texto.

Puedes consultar todos los modelos disponibles de sklear en el [user-guide](https://scikit-learn.org/stable/supervised_learning.html). 

En este proyecto puedes jugar con el preprocesamiento de datos, especificamente en la forma en que eliges representar el vector de texto. ¿quieres considerar si una palabra aparece en el vocabulario o cuantas veces lo hace? ¿Quieres excluir algunas palabras del vocabulario? etc. Considera que si la forma en que decidas hacerlo, deberás aplicar a los datos de validación durante inferencia **DE LA MISMA MANERA** en que preprocesaste los datos de entrenamiento. Modifica el método de `preprocess_dataset` y corre el notebook de nuevo para probar distintos experimentos.

Considera que **en todo momento** los datos de validación no se usan para encontrar ningún parámetro. Tienes que asumir que no existen hasta el momento que quieras predecir datos usando los modelos que hayas estimado con los datos de entrenamiento.


In [ ]:
from sklearn.linear_model import LogisticRegression

def train(X, label, model_type:str):
    data = X

    # TODO: Entrena el modelo y regresa el modelo entrenado en los datos de entrenamiento
    # model puede ser tanto la instancia de la clase que quieras usar, como un string indicando

    return estimator

def inference(modelo, X_val):
    # En inferencia, podemos recibir un solo dato entonces X_val.shape seria (D, )
    # Las clases de sklearn siempre esperan todo en la forma de  N, D
    if X_val.ndim == 1:
        X_val = X_val.reshape(1, -1)
    # Normalizamos los datos de validación
    # El mismos preprocesamiento de datos se aplica a
    # tanto inferencia como entrenamiento
    data = X_val

    # TODO: Utiliza el modelo para predecir valores para los datos de validación
    # Regresa las predicciones de tu modelo para X_val
    # En este caso, modelo tiene que ser una instancia de una clase para la cual quieres hacer predicción

    return preds

trained_models = {
    "algun_modelo": None,
    "otra_cosa": None,
    ...
}
for model_type in trained_models.keys():
    modelo = train(data_train, target_train, model_type=model_type)
    trained_models[model_type] = modelo

# 3. Evaluación y análisis de las predicciones
En esta sección incluimos funciones que te permiten visualizar la predicción de tu modelo para el set de validación. Dado que nuestros datos son de alta dimensionalidad (64) necesitamos reducirlos para poder analizar las predicciones. Recuerda que en esta sección solo funcionará si has definido tu modelo correctamente en el método anterior `mi_modelo`.

## 3.1 (Inferencia) Datos de validación en baja dimensionalidad
Completa el código de la siguiente celda para visualizar **las predicciones de TU modelo** de el conjunto de validación en baja dimensionalidad. Utiliza el método de reducción de dimensionalidad que consideres te ayude mejor a analizar tus datos. Cada clase/grupo deberá mostrarse en un color diferente. En base a lo que puedes observar en la imagen, ¿consideras que tu algoritmo ha aprendido algo que tiene sentido?

In [ ]:
def vis_low_dim(data_val, preds, model_type):
    fig, ax = plt.subplots(1, 1, figsize=(4,4))
    fig.suptitle(f"Puntos clasificados {model_type} (2 dimensiones)")
    n_groups = 2
    # Graficamos los datos, con un color diferente para cada clase/grupo
    print(f"Datos {data_val.shape}, predicciones {preds.shape}, clases/grupos {n_groups}")

    # TODO: Reduce los datos de VALIDACIÓN data_val a dos dimensiones para poder visualizarlos
    reduced_data = ...
    for g in range(n_groups):
        # TODO: Grafica los datos de VALIDACIÓN reducidos (reduced_data.shape = (N, 2))
        # Tal que grafiques aquellos que correspondan al grupo/clase group
        # Investiga plt.scatter, np.where o cómo filtrar arreglos dada una condición booleana
        ...
    fig.show()
    fig.legend()

## 3.2 (Inferencia) Análisis cualitativo
Completa el código de la siguiente celda. El siguiente código llama al método de inferencia anteriormente definido. Para cada modelo encuentra e imprime:
- 2 ejemplos de reseñas falsas positivas
- 2 ejemplos de reseñas falsas negativas
- 2 ejemplos de reseñas verdaderas positivas
- 2 ejemplos de reseñas verdaderas negativas

#### Métodos de clasificación
Si utilizaste un método de clasificación multiclase, los esperable sería que el valor real de la muestra (GT) sea igual al valor de la predicción para al menos la mayoría de los casos.

#### Métodos de agrupamiento
Si utilizaste un algoritmo de agrupamiento, es esperable que el valor real de la muestra (GT) no sea igual al grupo de tu predicción. Recuerda que al ser aprendizaje no supervisado, necesitamos adicionalmente "mapear" los grupos que haya encontrado el algoritmo a los reales. Puedes usar esta sección para hacer dicho mapeo. Lo mas sencillo es usar un diccionario

In [ ]:
data_val, target_val = preprocess_dataset(validation_set, get_vocab(training_set))
for name, trained_model in trained_models.items():
    # TODO: Para cada modelo, imprime los primeros 2 falsos positivos, 2 falsos negativos, 2 verdaderos positivos y 2 verdaderos negativos
    # Revisa el método de 'print_samples' para entender cómo imprimir los textos
    # Tip: Investiga el uso y funcionamiento de np.where
    preds = inference(modelo, data_val)
    fp_idcs = np.where(...)[0]
    fn_idcs = ...
    vp_idcs = ...
    vn_idcs = ...
    for idcs in [fp_idcs, fn_idcs, vp_idcs, vn_idcs]:
        samples = [validation_set[int(i)]['text'] for i in idcs]
        split_labels = ...
        split_preds = ...
        



## 3.2 (Inferencia) Análisis cuantitativo: Comparar rendimento de distintos modelos
En esta sección evalúa tus dos modelos entrenados en el conjunto de validación utilizando alguna métrica vista en clase (accuracy, F1, Precision, Recall etc.) y determina cuantitativamente cual funciona mejor. Investiga como usar las métricas de sklearn en la sección de [Classification metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics)

In [ ]:
from sklearn import metrics

# TODO: Para todos los modelos que entrenaste, calcula un valor 
# que indique la calidad de las predicciones en los datos de validación
# utiliza: data_val y target_val
for name, trained_model in trained_models.items():
    # Calcula la predicción y evalúa la calidad de predicción vs. las etiquetas reales (target_val)
    print(f"Modelo {name}: {score}")

En base al análisis tanto cualitativo como cuantitativo, discute en tu blog cual modelo funciona mejor justificando tu razonamiento. Puedes usar las siguientes preguntas como guía según las decisiones que hayas tomado:
- ¿Funcionó mejor entrenar en alta o baja dimensionalidad?
- ¿Funcionó mejor usar un método de aprendizaje supervisado o no supervisado?
- ¿Probaste algún método de preprocesamiento distinto?